In [ ]:
!pip install cvxpy
!pip install sympy SumOfSquares

In this notebook, we search for a Lyapunov function $V(\boldsymbol{x})$ that certifies the stability of the nonlinear system:

$$
\begin{align*}
  \dot{x}_1 &= -x_1 - 2 x_2^2 ,  \\
  \dot{x}_2 &= -x_2 - x_1 x_2 - 2 x_2^3 .
\end{align*}
$$

We parameterize the Lyapunov function as

$$
V(\boldsymbol{x}) = \frac{1}{2} \boldsymbol{x}^\top \underbrace{\begin{bmatrix} a_{11} & a_{21} \\ a_{21} & a_{22} \end{bmatrix}}_{\mathbf{A}} \boldsymbol{x} .
$$

The time-derivative of the Lyapunov function is

$$
\dot{V}(\boldsymbol{x}) = \boldsymbol{x}^\top \mathbf{A} \dot{\boldsymbol{x}} .
$$

For Lyapunov stability, $V(\boldsymbol{x})$ must be positive-definite, and $\dot{V}(\boldsymbol{x})$ must be negative-definite.

### Using semi-definite programming (SDP)

In [ ]:
import cvxpy as cp

A = cp.Variable((2, 2), symmetric=True)
B = cp.Variable((4, 4), symmetric=True)

constraints = [
    A >> 0,
    B >> 0,
    B[0,0] == A[0,0],
    B[1,0] == A[1,0],
    B[1,1] == A[1,1],
    B[2,0] == 0.5 * A[1,0],
    B[2,1] + B[3,0] == A[0,0] + 0.5 * A[1,1],
    B[2,2] == 0,
    B[3,1] == A[1,0],
    B[3,2] == 2 * A[1,0],
    B[3,3] == 2 * A[1,1],
    A[0,0] == 1, # fix a_11=1
]

problem = cp.Problem(cp.Minimize(0), constraints)
problem.solve(solver=cp.CLARABEL)

print("Status: ", problem.status)
print("A:\n", A.value)

Status:  optimal
A:
 [[ 1.00000000e+00 -2.14227498e-19]
 [-2.14227498e-19  1.99923116e+00]]


### Using sum of squares (SOS) programming

In [ ]:
import sympy as sp
from SumOfSquares import SOSProblem

x1, x2 = sp.symbols('x_1 x_2')
x = sp.Matrix([[x1], [x2]])

x1_dot = -x1 - 2 * x2**2
x2_dot = -x2 - x1 * x2 - 2 * x2**3
x_dot = sp.Matrix([[x1_dot], [x2_dot]])

a11, a21, a22 = sp.symbols('a_11 a_21 a_22')
A = sp.Matrix([
    [a11, a21],
    [a21, a22]])

V = 0.5 * x.T @ A @ x
V_dot = x.T @ A @ x_dot

prob = SOSProblem()
prob.add_sos_constraint(V[0], [x1, x2])
prob.add_sos_constraint(-V_dot[0], [x1, x2])
prob.add_constraint(prob.sym_to_var(a11) == 1) # fix a_11=1

prob.solve()

print("Status: ", problem.status)
print("A:\n", [[prob.sym_to_var(a11).value, prob.sym_to_var(a21).value],
               [prob.sym_to_var(a21).value, prob.sym_to_var(a22).value]])

Status:  optimal
A:
 [[0.9999999999999998, -6.620071977874407e-07], [-6.620071977874407e-07, 1.9774263847766123]]
